# Task 1: Data Collection and Preparation

### Imports:

In [38]:
import pandas as pd
import numpy as np
import re
import datetime as dt #for working with times objects

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

### LA set with all given data:

#### Reading in the raw data:

In [39]:
# get data from la 2019

la_set = pd.read_csv("la_2019.csv", low_memory = False)

#converting start and end time into timestamps

la_set['start_time'] = pd.DatetimeIndex(la_set['start_time'])
la_set['end_time'] = pd.DatetimeIndex(la_set['end_time'])

In [40]:
#adding the exact_duration to the dataframe
la_set["exact_duration"] = la_set["end_time"]-la_set["start_time"]

la_set

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
290337,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,4491,5903,Monthly Pass,Main & Winston,Main & Winston,1 days 17:37:59
290338,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
290339,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
290340,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


#### remove virtual station:

In [41]:
# remove virtual station VS (ID 3000)
la_set_dropVS = la_set[(la_set["start_station_id"] != 3000) & (la_set["end_station_id"] != 3000)]
la_set_dropVS


,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
290337,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,4491,5903,Monthly Pass,Main & Winston,Main & Winston,1 days 17:37:59
290338,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
290339,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
290340,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


In [42]:
#how many lines were dropped
lines_dropped = len(la_set)-len(la_set_dropVS)
print("{} lines were dropped.".format(lines_dropped))

4664 lines were dropped.


#### remove out of service stations:

In [43]:
# start end-stations "Metro Bike Share Free Bikes" ?? station 4285

# drop cells where station out of service OOS

la_set_dropOoS = la_set_dropVS[(la_set_dropVS["start_station_id"] != 4285)
                               & (la_set_dropVS["end_station_id"] != 4285)]

la_set_dropOoS.reset_index(drop=True, inplace=True)

la_set_dropOoS

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
279466,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,4491,5903,Monthly Pass,Main & Winston,Main & Winston,1 days 17:37:59
279467,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
279468,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
279469,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


In [44]:
#how many lines were dropped
lines_dropped = len(la_set_dropVS)-len(la_set_dropOoS)
print("{} lines were dropped.".format(lines_dropped))

6207 lines were dropped.


#### remove trips that longer 24 hours:

In [45]:
# Trips over 24 hours (long trips)

# drop cells where duration is over 24h


#more complex but unnessesary
la_set_dropLt = la_set_dropOoS[la_set_dropOoS["exact_duration"].apply(lambda x: x.total_seconds()) < (24*60*60)]

#[(la_set_dropOoS["exact_duration"].total_seconds())  > (24*60*60)]
la_set_dropLt.reset_index(drop=True, inplace=True)
la_set_dropLt

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
279017,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,3069,12019,Walk-up,Pershing Square,Broadway & 3rd,0 days 00:07:42
279018,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
279019,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
279020,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


In [46]:
#how many lines were dropped
lines_dropped = len(la_set_dropOoS)-len(la_set_dropLt)
print("{} lines were dropped.".format(lines_dropped))

449 lines were dropped.


#### remove short round trips:

In [47]:
# same start and  end-stations and duration 5min or below
# drop cells where are short round trips(under 5 minutes)

la_set_dropSrt = la_set_dropLt[((la_set_dropLt["start_station_id"] == la_set_dropLt["end_station_id"])
                               & (la_set_dropLt["exact_duration"].apply(lambda x: x.total_seconds()) > (5*60)))
                               | (la_set_dropLt["start_station_id"] != la_set_dropLt["end_station_id"])]
la_set_dropSrt.reset_index(drop=True, inplace=True)
la_set_dropSrt

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
270650,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,3069,12019,Walk-up,Pershing Square,Broadway & 3rd,0 days 00:07:42
270651,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
270652,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
270653,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


In [48]:
#how many lines were dropped
lines_dropped = len(la_set_dropLt)-len(la_set_dropSrt)
print("{} lines were dropped.".format(lines_dropped))

8367 lines were dropped.


#### remove trips under 1min:

In [49]:
# trips under 1min
# drop cells where exact_duration is under 1min
la_set_dropOm = la_set_dropSrt[la_set_dropSrt["exact_duration"].apply(lambda x: x.total_seconds()) > 59]

#how many lines were dropped
lines_dropped = len(la_set_dropSrt)-len(la_set_dropOm)
print("{} lines were dropped.".format(lines_dropped))
print(len(la_set_dropOm))

20 lines were dropped.
270635


#### remove rows with null values:

In [50]:
#rows with null values
# drop rows which have null values

la_set_dropNv = la_set_dropOm.dropna(axis=0, inplace=False)
la_set_dropNv

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
270650,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,3069,12019,Walk-up,Pershing Square,Broadway & 3rd,0 days 00:07:42
270651,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
270652,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
270653,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


#### remove test users:

In [51]:
#remove testing

la_set_dropTst = la_set_dropNv[(la_set_dropNv["user_type"] != "Testing")]

#how many lines were dropped
lines_dropped = len(la_set_dropNv)-len(la_set_dropTst)
print("{} lines were dropped.".format(lines_dropped))
print(len(la_set_dropOm))

29 lines were dropped.
270635


#### How many lines were dropped in total:

In [52]:
#how many lines were dropped
lines_dropped = len(la_set)-len(la_set_dropTst)
print("{} lines were dropped.".format(lines_dropped))

19736 lines were dropped.


#### Saving the cleaned Dataframe in the original la_set:

In [53]:
# assign la_set to dataset where Out Of Service and Virtual Station are removed
la_set = la_set_dropNv
#reset indices
la_set.reset_index(drop=True)
la_set

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,exact_duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,6468,Walk-up,2nd & Hill,7th & Broadway,0 days 00:07:00
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,0 days 00:06:00
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,5992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,5860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,6006,Walk-up,Main & 1st,Broadway & 9th,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...
270650,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,3069,12019,Walk-up,Pershing Square,Broadway & 3rd,0 days 00:07:42
270651,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,0 days 00:09:06
270652,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,0 days 00:04:22
270653,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,0 days 00:05:38


#### Saving the Datasets in the folder, so the other notebooks can use them:

In [54]:
la_set.to_pickle("la_set.pkl")